In [ ]:
%reload_ext autoreload
%autoreload 2

import time
import itertools
import numpy as np
import matplotlib.pyplot as plt

import tqdm 
import tqdm.notebook

from optimization import G1, G2
from optimization_clean import *
import multiprocessing


In [ ]:
def plotResult(selected_cities_n, loss_values, num_cities_per_step):
    print('Final loss ', loss_values[-1])
    fig, axes = plt.subplots(1, 2, figsize=(12, 4))
    axes[0].plot(loss_values)
    axes[0].set_ylabel('Loss')
    axes[0].set_xlabel('Iterations')
    axes[0].set_title('Loss Evolution')
    m = selected_cities_n == 1
    not_selected = selected_cities_n == 0
    axes[1].scatter(g.x[not_selected, 0], g.x[not_selected, 1],
                    label='Non selected Cities')
    axes[1].scatter(g.x[m, 0], g.x[m, 1], c='r', label='Selected cities')
    axes[1].set_title('Selected cities')
    box = axes[1].get_position()
    axes[1].set_position([box.x0, box.y0, box.width * 0.8, box.height])
    # Put a legend to the right of the current axis
    axes[1].legend(loc='center left', bbox_to_anchor=(1, 0.5))

    plt.xlim(0, 1)
    plt.ylim(0, 1)
    if num_cities_per_step is not None:
        plt.figure(figsize=(4, 2))
        plt.plot(np.arange(n_iter), num_cities_per_step)
        plt.title("#selected cities in each step")

In [ ]:
# Here we would load a given data set

# np.random.seed(42)
N = 1000
g = G2(N)

l = 1.5 # lambda


In [ ]:
# Run the optimization to compute the selected cities
n_iter = 20000

def beta(i):
    p1 = 0.5
    p50 = 0.006
    beta0 = -np.log(p1)
    beta1 = -np.log(p50)
    v = beta0 * (beta1 / beta0) ** (i/(n_iter-1.0)) 
    return v *4
    
def beta(i):
    if i < n_iter // 2: 
        return 0.4
    elif i < n_iter * 2/ 3:
        return 1
    else: 
        return 5
    
    
def run_optimization(i):
    np.random.seed(i)
    curr_selected, curr_loss_values = optimize(g, l, beta=beta, n_iter=n_iter, verbose=True)
    return curr_selected, curr_loss_values

use_multithreading = True
losses = []
all_selected = []
total_runs = 12
if use_multithreading:
    with multiprocessing.Pool(12) as p:
        with tqdm.notebook.tqdm(range(total_runs)) as pbar:
            for i, result in enumerate(p.imap_unordered(run_optimization, range(total_runs))):
                selected, loss_values = result
                pbar.update()
                losses.append(loss_values)
                all_selected.append(selected)
else:
    for i in tqdm.notebook.tqdm(range(total_runs)):
        selected, loss_values = run_optimization(i)
        losses.append(loss_values)
        all_selected.append(selected)

losses = np.array(losses)
print(losses[:, -1])
# num_cities_per_step = np.array(selected_cities_n).sum(axis=1)
# if type(selected_cities_n) == list:
#     selected_cities_n = selected_cities_n[-1]

# plotResult(selected_cities_n, loss_values, num_cities_per_step)


In [ ]:
# min_idx = -1
# min_loss_value = np.inf
# n_initializations = 20
# for i in range(n_initializations):
#     curr_selected, curr_loss_values = optimize(g, l, beta=beta, n_iter=n_iter)
#     # loss_convex_pass_one = loss_value_convex
#     # # Second pass
#     # if mutation_strategy == 5: 
#     #     selected_cities_n, selected_cities_n_convex, loss_values, loss_value_convex = optimize(
#     #         g, l, 
#     #         beta=35, 
#     #         n_iter=n_iter, 
#     #         mutation_strategy=4,
#     #         initial_selection_probability=initial_selection_probability,
#     #         selected_cities=selected_cities_n_convex)
#     #     print(f"Second pass changed loss from {loss_convex_pass_one} to {loss_value_convex}")
#     print(curr_loss_values[-1])
#     if curr_loss_values[-1] < min_loss_value: 
#         min_loss_value = curr_loss_values[-1]
#         loss_values = curr_loss_values
#         selected_cities_n = curr_selected

# num_cities_per_step = np.array(selected_cities_n).sum(axis=1)
# if type(selected_cities_n) == list:
#     selected_cities_n = selected_cities_n[-1]

# plotResult(selected_cities_n, loss_values, num_cities_per_step)